In [ ]:
# 导入包
from shapely.geometry import Polygon, MultiPolygon
from matplotlib import cm
import matplotlib
from getchinamap.getchinamap import DownloadChmap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs 
import platform
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import geopandas as gpd
import cartopy.feature
from cartopy.mpl.patch import geos_to_path

import itertools

from mpl_toolkits.mplot3d import Axes3D
# import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection,PolyCollection

# matplotlib 显示中文的问题
if platform.system() == 'Darwin':
    plt.rcParams["font.family"] = 'Arial Unicode MS'
elif platform.system() == 'Windows':
    plt.rcParams["font.family"] = 'SimHei'
else:
    pass

In [ ]:
chinamapdata = gpd.read_file("https://geo.datav.aliyun.com/areas_v3/bound/geojson?code=100000_full")
chinamapdata.head(3)

In [ ]:
sampledata = pd.DataFrame({'lon':chinamapdata.centroid.x, 'lat':chinamapdata.centroid.y, 'name':chinamapdata['name']})
sampledata['value'] = np.random.randint(0, 1000, sampledata.shape[0])
sampledata.head(4)

In [ ]:
chinamapdata.crs

In [ ]:
bounds_box = chinamapdata.bounds
minx = bounds_box['minx'].min()
miny = bounds_box['miny'].min()
maxx = bounds_box['maxx'].max()
maxy = bounds_box['maxy'].max()

In [ ]:


geoms = chinamapdata.geometry
# target_projection = ccrs.PlateCarree()
# geoms = [target_projection.project_geometry(geom, target_projection)
#          for geom in geoms]

paths = list(itertools.chain.from_iterable(geos_to_path(geom) for geom in geoms))

# At this point, we start working around mpl3d's slightly broken interfaces.
# So we produce a LineCollection rather than a PathCollection.
segments = []
for path in paths:
    vertices = [vertex for vertex, _ in path.iter_segments()]
    vertices = np.asarray(vertices)
    segments.append(vertices)

lc = LineCollection(segments, color='black')


In [ ]:
%matplotlib widget

In [ ]:
# part 1
segments = []
for path in paths:
    vertices = [vertex for vertex, _ in path.iter_segments()]
    vertices = np.asarray(vertices)
    segments.append(vertices)

# with plt.style.context('fivethirtyeight'):

fig = plt.figure()
ax = Axes3D(fig, xlim=[minx, maxx], ylim=[miny, maxy])
# ax.set_zlim(bottom=0)

lc = LineCollection(segments, color='black',linewidths=1)

ax.add_collection3d(lc)


ax.bar3d(x=sampledata['lon'], y=sampledata['lat'], z=np.zeros_like(sampledata['value']),
        dx=np.ones_like(sampledata['value']), 
        dy=np.ones_like(sampledata['value']), 
        dz=sampledata['value'],alpha=0.8)

# for index, iterrow in sampledata.iterrows():
#     ax.text(iterrow['lon'], iterrow['lat'], iterrow['value']+2,iterrow['name'], color='green',size=9)

ax.text(80, 30, 1000, '公众号: world of statistics', size=20, color='gray')

ax.set_xlabel('经度')
ax.set_ylabel('维度')
# ax.set_xlim([minx, maxx])
# ax.set_ylim([miny, maxy])
ax.set_zlabel('value')
    # ax.set_title("公众号: world of statistics")


plt.show()


In [ ]:
# geoms
# part 2

fig = plt.figure()
ax = Axes3D(fig, xlim=[minx, maxx], ylim=[miny, maxy])


concat = lambda iterable: list(itertools.chain.from_iterable(iterable))
polys = concat(path.to_polygons() for path in paths)

lc = PolyCollection(polys, edgecolor='black',
                    facecolor='green', closed=False, alpha=0.4)

ax.add_collection3d(lc)

ax.bar3d(x=sampledata['lon'], y=sampledata['lat'], z=np.zeros_like(sampledata['value']),
        dx=np.ones_like(sampledata['value']), 
        dy=np.ones_like(sampledata['value']), 
        dz=sampledata['value'],alpha=0.8)


ax.text(80, 30, 1000, '公众号: world of statistics', size=20, color='gray')

ax.set_xlabel('经度')
ax.set_ylabel('维度')
# ax.set_xlim([minx, maxx])
# ax.set_ylim([miny, maxy])
ax.set_zlabel('value')
    # ax.set_title("公众号: world of statistics")


plt.show()

### 参考链接
1. https://pythonprogramming.net/3d-bar-chart-matplotlib-tutorial/
2. https://stackoverflow.com/questions/23785408/3d-cartopy-similar-to-matplotlib-basemap
3. https://matplotlib.org/stable/gallery/mplot3d/text3d.html
4. https://datav.aliyun.com/portal/school/atlas/area_selector
5. https://matplotlib.org/stable/gallery/mplot3d/3d_bars.html
6. https://stackoverflow.com/questions/19390895/matplotlib-plot-with-variable-line-width